<a href="https://colab.research.google.com/github/AlanMarquesRocha/pv_cells_detection_fault/blob/main/pv_cells_detection_fault.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Informações a cerca da base de dados do projeto:

A base de dados contém **2.624 imagens de células FVs de 300 x 300 pixels em níveis de cinza (8 bits)**, sendo 1.074 imagens de células de silício monocristalino e 1.550 imagens de células de silício policristalino, extraídas de 44 módulos FVs diferentes.

Cada imagem foi rotulada com um número de ponto flutuante que caracteriza a probabilidade de defeito da mesma, podendo variar entre 0 e 1, onde:

0 - célula FV sem probabilidade de defeito
1 - célula FV com alto grau de probabilidade de defeito.

As células são divididas em ``mono`` e ``poly``, representando as células de silício monocristalino e policristalino respectivamente.

Pode-se verificar qual tipo de célula estamos trabalhando através do arquivo ``labels.csv``. Neste arquivo, constam as seguintes informações:

- nome da imagem (Ex: cell236.png)
- grau de defeito (0.0, 1.0, 0.333, 0.666)
- tipo de célula (mono ou poly)

### Autores que disponibilizaram a base (Citação):

> Buerhop-Lutz, C.; Deitsch, S.; Maier, A.; Gallwitz, F.; Berger, S.; Doll, B.; Hauch, J.; Camus, C. & Brabec, C. J. A Benchmark for Visual Identification of Defective Solar Cells in Electroluminescence Imagery. European PV Solar Energy Conference and Exhibition (EU PVSEC), 2018. DOI: [10.4229/35thEUPVSEC20182018-5CV.3.15](http://dx.doi.org/10.4229/35thEUPVSEC20182018-5CV.3.15)

> Deitsch, S., Buerhop-Lutz, C., Sovetkin, E., Steland, A., Maier, A., Gallwitz, F., & Riess, C. (2021). Segmentation of photovoltaic module cells in uncalibrated electroluminescence images. Machine Vision and Applications, 32(4). DOI: [10.1007/s00138-021-01191-9](https://doi.org/10.1007/s00138-021-01191-9)

> Deitsch, S.; Christlein, V.; Berger, S.; Buerhop-Lutz, C.; Maier, A.; Gallwitz, F. & Riess, C. Automatic classification of defective photovoltaic module cells in electroluminescence images. Solar Energy, Elsevier BV, 2019, 185, 455-468. DOI: [10.1016/j.solener.2019.02.067](http://dx.doi.org/10.1016/j.solener.2019.02.067)

BibTeX details:

<details>

```bibtex

@InProceedings{Buerhop2018,
  author    = {Buerhop-Lutz, Claudia and Deitsch, Sergiu and Maier, Andreas and Gallwitz, Florian and Berger, Stephan and Doll, Bernd and Hauch, Jens and Camus, Christian and Brabec, Christoph J.},
  title     = {A Benchmark for Visual Identification of Defective Solar Cells in Electroluminescence Imagery},
  booktitle = {European PV Solar Energy Conference and Exhibition (EU PVSEC)},
  year      = {2018},
  eventdate = {2018-09-24/2018-09-28},
  venue     = {Brussels, Belgium},
  doi       = {10.4229/35thEUPVSEC20182018-5CV.3.15},
}

@Article{Deitsch2021,
  author       = {Deitsch, Sergiu and Buerhop-Lutz, Claudia and Sovetkin, Evgenii and Steland, Ansgar and Maier, Andreas and Gallwitz, Florian and Riess, Christian},
  date         = {2021},
  journaltitle = {Machine Vision and Applications},
  title        = {Segmentation of photovoltaic module cells in uncalibrated electroluminescence images},
  doi          = {10.1007/s00138-021-01191-9},
  issn         = {1432-1769},
  number       = {4},
  volume       = {32},
}

@Article{Deitsch2019,
  author    = {Sergiu Deitsch and Vincent Christlein and Stephan Berger and Claudia Buerhop-Lutz and Andreas Maier and Florian Gallwitz and Christian Riess},
  title     = {Automatic classification of defective photovoltaic module cells in electroluminescence images},
  journal   = {Solar Energy},
  year      = {2019},
  volume    = {185},
  pages     = {455--468},
  month     = jun,
  issn      = {0038-092X},
  doi       = {10.1016/j.solener.2019.02.067},
  publisher = {Elsevier {BV}},
}
```

### **Importação das bibliotecas necessárias para o projeto**

In [1]:
# Bibliotecas necessárias para o projeto (Importadas a medida que são utilizadas)

from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2


### **Definindo os métodos para abertura do dataset**

In [2]:

# Função que carrega o dataset completo diretamente do diretório raiz.

def load_dataset(fname = None):
    if fname is None:
        fname = os.path.abspath(os.path.join(os.path.dirname("elpv-dataset-master\elpv-dataset-master"),
                                             os.path.pardir))
        fname = os.path.join(fname, 'elpv-dataset-master\labels.csv')

    data = np.genfromtxt(fname, dtype=['|S19', '<f8', '|S4'], names=[
                         'path', 'probability', 'type'])
    image_fnames = np.char.decode(data['path'])
    probs = data['probability']
    types = np.char.decode(data['type'])

    def load_cell_image(fname):
        with Image.open(fname) as image:
            return np.asarray(image)

    dir = os.path.dirname(fname)
    
    images = np.array([load_cell_image(os.path.join(dir, fn))
                       for fn in image_fnames])

    return images, probs, types

In [3]:


# Função que carrega o dataset de células FV monocristalinas diretamente do diretório raiz.

def load_dataset_mono(fname_mono = None):
    if fname_mono is None:
        fname_mono = os.path.abspath(os.path.join(os.path.dirname("elpv-dataset-master\elpv-dataset-master"),
                                             os.path.pardir))
        fname_mono = os.path.join(fname_mono, 'elpv-dataset-master\labels_mono.csv')

    data_mono = np.genfromtxt(fname_mono, dtype=['|S19', '<f8', '|S4'], names=[
                         'path', 'probability', 'type'])
    image_fnames_mono = np.char.decode(data_mono['path'])
    probs_mono = data_mono['probability']
    types_mono = np.char.decode(data_mono['type'])

    def load_cell_image_mono(fname_mono):
        with Image.open(fname_mono) as image_mono:
            return np.asarray(image_mono)

    dir = os.path.dirname(fname_mono)
    
    images_mono = np.array([load_cell_image_mono(os.path.join(dir, fn))
                       for fn in image_fnames_mono])

    return images_mono, probs_mono, types_mono

In [4]:

#Função que carrega o dataset de células FV polycristalinas diretamente do diretório raiz.

def load_dataset_poly(fname_poly = None):
    if fname_poly is None:
        fname_poly = os.path.abspath(os.path.join(os.path.dirname("elpv-dataset-master\elpv-dataset-master"),
                                             os.path.pardir))
        fname_poly = os.path.join(fname_poly, 'elpv-dataset-master\labels_poly.csv')

    data_poly = np.genfromtxt(fname_poly, dtype=['|S19', '<f8', '|S4'], names=[
                         'path', 'probability', 'type'])
    image_fnames_poly = np.char.decode(data_poly['path'])
    probs_poly = data_poly['probability']
    types_poly = np.char.decode(data_poly['type'])

    def load_cell_image_poly(fname_poly):
        with Image.open(fname_poly) as image_poly:
            return np.asarray(image_poly)

    dir = os.path.dirname(fname_poly)
    
    images_poly = np.array([load_cell_image_poly(os.path.join(dir, fn))
                       for fn in image_fnames_poly])

    return images_poly, probs_poly, types_poly

In [7]:

# Chamando a função para leitura das imagens do dataset completo.
images, proba, types = load_dataset()

# Chamando a função para leitura das imagens do dataset de células monocristalinas [1 - 1073].
images_mono, proba_mono, types_mono = load_dataset_mono()

# Chamando a função para leitura das imagens do dataset de células policristalinas [1 - 1549].
images_poly, proba_poly, types_poly = load_dataset_poly()

OSError: ignored

In [6]:
img = images_mono[10]

NameError: ignored